<a href="https://colab.research.google.com/github/CleideLustosa/AtividadeEntregaLogProgTicTrilha/blob/main/Ex_mlp_penguins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg"  width=300, align="right">


# Lab Multi Layer Perceptron

Neste exercício você vai empregar redes MLP do `scikit-learn` para resolver um problema de classificação sobre a base de dados `penguins`. Siga o modelo fornecido.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Modelo MLP

In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

## Preparação dos dados



In [3]:
# Carregando os dados
df = pd.read_csv('https://vincentarelbundock.github.io/Rdatasets/csv/MASS/Cars93.csv',index_col=0)
df = df.reset_index(drop=True)

### Dropna

Ou algum outro tratamento para valores ausentes.

In [4]:
df.isna().sum()

,0
Manufacturer,0
Model,0
Type,0
Min.Price,0
Price,0
Max.Price,0
MPG.city,0
MPG.highway,0
AirBags,34
DriveTrain,0


Não vamos empregar esses dados com valores ausentes, então, não é necessário excluir as linhas.

In [5]:
# Separando preditores e classe objetivo
X = df[['MPG.city', 'Horsepower', 'Type', 'Price', 'Width', 'Length', 'Weight', 'EngineSize', 'DriveTrain']]
y = df['Origin']

display(pd.concat([X,y],axis=1).head())

# Separando os dados de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=1)

,MPG.city,Horsepower,Type,Price,Width,Length,Weight,EngineSize,DriveTrain,Origin
0,25,140,Small,15.9,68,177,2705,1.8,Front,non-USA
1,18,200,Midsize,33.9,71,195,3560,3.2,Front,non-USA
2,20,172,Compact,29.1,67,180,3375,2.8,Front,non-USA
3,19,172,Midsize,37.7,70,193,3405,2.8,Front,non-USA
4,22,208,Midsize,30.0,69,186,3640,3.5,Rear,non-USA


### Hot encode dos dados

Assim como na normalização, fazer o `fit_transform()` nos dados de treinamento e a apenas o `transform()` nos dados de teste para evitar o *leak* de informações para o treinamento.

In [6]:
# Aplicando one-hot encode
encoder = OneHotEncoder(sparse_output=False)
X_train_encoded = encoder.fit_transform(X_train[['Type', 'DriveTrain']])
X_train_encoded = pd.DataFrame(X_train_encoded, columns=encoder.get_feature_names_out())
X_train = pd.concat([X_train.drop(columns=['Type', 'DriveTrain']).reset_index(drop=True), X_train_encoded], axis=1)
display(X_train.head())

X_test_encoded = encoder.transform(X_test[['Type', 'DriveTrain']])
X_test_encoded = pd.DataFrame(X_test_encoded, columns=encoder.get_feature_names_out())
X_test = pd.concat([X_test.drop(columns=['Type', 'DriveTrain']).reset_index(drop=True), X_test_encoded], axis=1)
display(X_test.head())

,MPG.city,Horsepower,Price,Width,Length,Weight,EngineSize,Type_Compact,Type_Large,Type_Midsize,Type_Small,Type_Sporty,Type_Van,DriveTrain_4WD,DriveTrain_Front,DriveTrain_Rear
0,26,164,16.5,69,184,2970,2.5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,18,202,26.1,70,190,3730,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,19,170,24.4,74,177,3495,3.8,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,17,151,19.1,74,190,4100,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,33,73,8.4,60,146,2045,1.2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


,MPG.city,Horsepower,Price,Width,Length,Weight,EngineSize,Type_Compact,Type_Large,Type_Midsize,Type_Small,Type_Sporty,Type_Van,DriveTrain_4WD,DriveTrain_Front,DriveTrain_Rear
0,17,255,32.5,69,169,2895,1.3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,30,90,12.5,67,164,2475,1.6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,29,110,11.8,66,170,2545,1.6,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,19,200,18.5,72,195,3450,3.4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,23,110,11.1,66,181,2575,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Normalizando os dados

Vamos adotar a normalização *z-score* (StandardScaler), $x_{scaled} = \frac{x - \bar{x}}{\sigma(x)}$

In [7]:
# Normalizando os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
display(pd.DataFrame(X_train).head())
display(pd.DataFrame(X_test).head())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.575704,0.432470,-0.308597,-0.108099,0.107916,-0.186301,-0.173359,2.345208,-0.374634,-0.523937,-0.571429,-0.400892,-0.374634,-0.400892,0.642685,-0.426401
1,-0.731572,1.162114,0.661014,0.152139,0.504966,1.055704,0.300101,-0.426401,-0.374634,1.908627,-0.571429,-0.400892,-0.374634,-0.400892,0.642685,-0.426401
2,-0.568162,0.547677,0.489312,1.193089,-0.355309,0.671663,1.057637,-0.426401,2.669270,-0.523937,-0.571429,-0.400892,-0.374634,-0.400892,0.642685,-0.426401
3,-0.894981,0.182854,-0.045994,1.193089,0.504966,1.660364,0.300101,-0.426401,-0.374634,-0.523937,-0.571429,-0.400892,2.669270,-0.400892,0.642685,-0.426401
4,1.719570,-1.314837,-1.126707,-2.450236,-2.406733,-1.697951,-1.404356,-0.426401,-0.374634,-0.523937,1.750000,-0.400892,-0.374634,2.494438,-1.555973,-0.426401


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,-0.894981,2.179776,1.307421,-0.108099,-0.884709,-0.308867,-1.309664,-0.426401,-0.374634,-0.523937,-0.571429,2.494438,-0.374634,-0.400892,-1.555973,2.345208
1,1.229342,-0.988417,-0.712602,-0.628574,-1.215583,-0.995238,-1.025588,-0.426401,-0.374634,-0.523937,-0.571429,2.494438,-0.374634,-0.400892,0.642685,-0.426401
2,1.065932,-0.604394,-0.783303,-0.888811,-0.818534,-0.880843,-1.025588,-0.426401,-0.374634,-0.523937,1.750000,-0.400892,-0.374634,-0.400892,0.642685,-0.426401
3,-0.568162,1.123712,-0.106595,0.672614,0.835841,0.598123,0.678869,-0.426401,-0.374634,1.908627,-0.571429,-0.400892,-0.374634,-0.400892,0.642685,-0.426401
4,0.085476,-0.604394,-0.854003,-0.888811,-0.090609,-0.831816,-0.646819,2.345208,-0.374634,-0.523937,-0.571429,-0.400892,-0.374634,-0.400892,0.642685,-0.426401


### Treinamento do Modelo

Definição do modelo e seus *Hiperparâmetros*, e o ajuste (treinamento) do Modelo.

In [8]:
# Define o modelo de classificação (MLP)
clf = MLPClassifier(hidden_layer_sizes=(8,16,8), random_state=1, max_iter=10000)

# Treinando o modelo
clf.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(8, 16, 8), max_iter=10000, random_state=1)

### Métricas de Eficiência do Modelo

Acuracidade, Classification Report, sobre o conjunto de teste.


In [9]:
# Calculando métricas de avaliação
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("Acurácia:", accuracy)
print("Classification Report:\n", classification_rep)


Acurácia: 0.6071428571428571
Classification Report:
               precision    recall  f1-score   support

         USA       0.62      0.57      0.59        14
     non-USA       0.60      0.64      0.62        14

    accuracy                           0.61        28
   macro avg       0.61      0.61      0.61        28
weighted avg       0.61      0.61      0.61        28



### Predição

Faça a predição dos seguintes veículos em `X_new`:

In [10]:
#@markdown Execute para gerar `X_new`
X_new = df[ df.Model.isin(['Mustang','Accord']) ][X.columns].reset_index(drop=True)



In [11]:
display(X_new)

,MPG.city,Horsepower,Type,Price,Width,Length,Weight,EngineSize,DriveTrain
0,22,105,Sporty,15.9,68,180,2850,2.3,Rear
1,24,140,Compact,17.5,67,185,3040,2.2,Front


**Atenção**: Antes de aplicar a predição é necessário aplicar as mesmas transformações empregadas antes durante o treinamento.

In [12]:
X_encoded = encoder.transform(X_new[['Type', 'DriveTrain']])
X_encoded = pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out())
X_new = pd.concat([X_new.drop(columns=['Type', 'DriveTrain']), X_encoded], axis=1)
display(X_new.head())

X_new = scaler.transform(X_new)

y_pred = clf.predict(X_new)

print(y_pred)

,MPG.city,Horsepower,Price,Width,Length,Weight,EngineSize,Type_Compact,Type_Large,Type_Midsize,Type_Small,Type_Sporty,Type_Van,DriveTrain_4WD,DriveTrain_Front,DriveTrain_Rear
0,22,105,15.9,68,180,2850,2.3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,24,140,17.5,67,185,3040,2.2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


['USA' 'non-USA']


# Exercício

Empregue os modelos de rede neural MLP abaixo para classificar a origem, ilha do Pinguim, baseado em todos os demais atributos.


In [13]:
df = sns.load_dataset('penguins')
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


## Preparação dos Dados

Exclua os dados ausentes se houverem, excluindo as linhas.


In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Remover linhas com valores ausentes
df = df.dropna()

# Separar features (X) e alvo (y)
X = df[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g', 'sex']]
y = df['island']

# Converta 'sex' para numérico se necessário
X = pd.get_dummies(X, columns=['sex'], drop_first=True)

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=1)


### Dropna

In [15]:
#@markdown Verificação, precisa retornar True
len(df) == 333

True

### Dados de treinamento e teste

Empregue 30% dos dados para teste, `random_state=1` e conjuntos estratificados pelo atributo classe (variável objetivo).

In [22]:
train, X_test = train_test_split(df, test_size=0.3, random_state=1, stratify=df['species'])
X_train = train.drop('species', axis=1)
y_train = train['species']
y_test = X_test['species']



In [23]:
#@markdown Verificação, precisa retornar True

len(X_test[X_test.species == 'Gentoo']) == 36

True

Divisão de Dados


In [75]:
from sklearn.model_selection import train_test_split

train, X_test = train_test_split(df, test_size=0.3, random_state=1, stratify=df['species'])
X_train = train.copy()




### Hot Encode



In [87]:
import pandas as pd

categorical_cols = [col for col in ['island', 'sex'] if col in X_train.columns]

X_train = pd.get_dummies(X_train, columns=categorical_cols, drop_first=False)
X_test = pd.get_dummies(X_test, columns=categorical_cols, drop_first=False)

species_dummies_train = pd.get_dummies(X_train['species'], prefix='species', drop_first=False)
species_dummies_test = pd.get_dummies(X_test['species'], prefix='species', drop_first=False)

X_train = pd.concat([X_train, species_dummies_train], axis=1)
X_test = pd.concat([X_test, species_dummies_test], axis=1)

X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

print(X_train.columns)








Index(['species', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm',
       'body_mass_g', 'island_Biscoe', 'island_Dream', 'island_Torgersen',
       'sex_Female', 'sex_Male', 'species_Adelie', 'species_Chinstrap',
       'species_Gentoo', 'species_Adelie', 'species_Chinstrap',
       'species_Gentoo', 'species_Adelie', 'species_Chinstrap',
       'species_Gentoo'],
      dtype='object')


In [77]:
#@markdown Verificação, precisa retornar True

X_train.species_Adelie.sum() == 102

np.True_

### Normalização




In [89]:
from sklearn.preprocessing import StandardScaler

# Features numéricas e dummies (EXCETO species que é target)
feature_cols = [col for col in X_train.columns if not col.startswith('species_') and col != 'species']

scaler = StandardScaler()
X_train_scaled_values = scaler.fit_transform(X_train[feature_cols])
X_test_scaled_values = scaler.transform(X_test[feature_cols])

# IMPORTANTE: Atualize X_train e X_test COM os valores normalizados!
X_train[feature_cols] = X_train_scaled_values
X_test[feature_cols] = X_test_scaled_values

# Agora X_test contém valores normalizados, mantendo a estrutura!




In [92]:
# Para treinar, use as features normalizadas (já estão em X_train/X_test)
feature_cols = [col for col in X_train.columns if not col.startswith('species_') and col != 'species']

X_train_model = X_train[feature_cols].values
X_test_model = X_test[feature_cols].values

# Labels separados
y_train = (X_train['species_Adelie'].astype(int)).values
y_test = (X_test['species_Adelie'].astype(int)).values


In [93]:
print("Coluna na posição 1:", X_test.columns[1])
print("Soma:", np.round(X_test.iloc[:, 1].sum(), 2))


Coluna na posição 1: bill_length_mm
Soma: 5.67


In [95]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(hidden_layer_sizes=(8, 16), max_iter=5000, random_state=1)
clf.fit(X_train_model, y_train)

y_pred = clf.predict(X_test_model)


In [96]:
#@markdown Verificação, precisa retornar True
np.round(X_test[1].sum(), 2) == -1.2

KeyError: 1

## Modelo MLP 1

Implemente um algoritmo MLP para classificação da ilha (`island`) de origem dos pinguins. Empregue `random_state=1`, uma rede com camadas ocultas de 8, 16 e 8 neurônios respectivamente e número máximo de iterações 5000. Não empregue outros parâmetros além dos solicitados.

Obtenha ao final a acuracidade do modelo, o `classification report` e a matriz de confusão.

In [100]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Use EXATAMENTE estes parâmetros (conforme a imagem solicitava)
clf = MLPClassifier(
    hidden_layer_sizes=(8, 16),
    max_iter=5000,
    random_state=1
)

# Treine com os dados normalizados
clf.fit(X_train_model, y_train)

# Faça predições
y_pred = clf.predict(X_test_model)

# Obtenha as métricas
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

# Verifique o valor dos coefs
print(f"Soma dos coefs[0]: {np.round(clf.coefs_[0].sum(), 2)}")


              precision    recall  f1-score   support

           0       0.98      0.98      0.98        44
           1       1.00      0.98      0.99        44
           2       1.00      0.98      0.99        44

   micro avg       0.99      0.98      0.98       132
   macro avg       0.99      0.98      0.98       132
weighted avg       0.99      0.98      0.98       132
 samples avg       0.43      0.43      0.43       132



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


ValueError: multilabel-indicator is not supported

In [99]:
#@markdown Verificação, precisa retornar True
np.round(clf.coefs_[0].sum(),2) == -3.77

np.False_

## Modelo MLP 2

Altere o modelo anterior para uma rede com 1000 e 500 elementos de camadas internas, a função de ativação `relu`. Matenha os demais parâmetros e empregue o mesmo conjunto de Treinamento e Teste do modelo anterior.

In [101]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Modelo MLP 2 com arquitetura diferente
clf2 = MLPClassifier(
    hidden_layer_sizes=(1000, 500),
    max_iter=5000,
    random_state=1
)

# Treine com os dados normalizados
clf2.fit(X_train_model, y_train)

# Faça predições
y_pred2 = clf2.predict(X_test_model)

# Obtenha as métricas
print(classification_report(y_test, y_pred2))
print(confusion_matrix(y_test, y_pred2))

# Verifique o valor dos coefs
print(f"Soma dos coefs[0]: {np.round(clf2.coefs_[0].sum(), 2)}")


              precision    recall  f1-score   support

           0       1.00      0.98      0.99        44
           1       1.00      0.98      0.99        44
           2       1.00      0.98      0.99        44

   micro avg       1.00      0.98      0.99       132
   macro avg       1.00      0.98      0.99       132
weighted avg       1.00      0.98      0.99       132
 samples avg       0.43      0.43      0.43       132



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


ValueError: multilabel-indicator is not supported

In [102]:
#@markdown Verificação, precisa retornar True
np.round(clf.coefs_[1].sum(),2) == 3.85

np.False_

## Predição

Faça a predição dos seguinte pinguim:

```
species                 Adelie
bill_length_mm            40.0
bill_depth_mm             18.0
flipper_length_mm        185.0
body_mass_g             3900.0
sex                       Male
```

In [104]:
# Versão simplificada se não souber a island
novo_pinguim = pd.DataFrame({
    'bill_length_mm': [40.0],
    'bill_depth_mm': [18.0],
    'flipper_length_mm': [185.0],
    'body_mass_g': [3900.0],
    'sex': ['Male']
})

# Crie as dummies de sex
novo_pinguim_encoded = pd.get_dummies(novo_pinguim, columns=['sex'], drop_first=False)

# Adicione colunas que podem estar faltando
for col in X_train_model.columns:
    if col not in novo_pinguim_encoded.columns:
        novo_pinguim_encoded[col] = 0

# Selecione apenas as features na ordem correta
novo_pinguim_scaled = scaler.transform(novo_pinguim_encoded[feature_cols])

# Predição
pred_class = clf2.predict(novo_pinguim_scaled)
pred_proba = clf2.predict_proba(novo_pinguim_scaled)

print(f"Classe predita: {pred_class}")
print(f"Probabilidades por classe: {pred_proba}")



AttributeError: 'numpy.ndarray' object has no attribute 'columns'

Qual a probabilidade de cada uma das classes na predição?

In [105]:
#@markdown Verificação, precisa retornar True
np.round(clf.predict_proba(X_new)[0][2],2) == 0.22

ValueError: X has 16 features, but MLPClassifier is expecting 9 features as input.